In [ ]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob
import os, os.path
import keras
import time as timer
import scipy.misc
import scipy.ndimage

from keras.models import load_model

from tqdm import tqdm
from datetime import datetime

from sklearn.metrics import roc_auc_score, roc_curve

%matplotlib inline

from datetime import datetime
from uncertainties import ufloat, unumpy
from uncertainties.umath import *

from PIL import Image

In [ ]:
# resolution of images

h = 75
w = 100

# Timing analysis

In [ ]:
img_neg_dark = np.array(Image.open('../../../WindingImages/2017-09-10/IMG_2017-09-10_15-56-08.jpg'))
plt.imshow(img_neg_dark, cmap='gray')

In [ ]:
img_neg_light = np.array(Image.open('../../../WindingImages/2017-09-10/IMG_2017-09-10_15-56-01.jpg'))
plt.imshow(img_neg_light, cmap='gray')

In [ ]:
img_pos_dark = np.array(Image.open('../../../WindingImages/2017-09-10/IMG_2017-09-10_15-55-20.jpg'))
plt.imshow(img_pos_dark, cmap='gray')

In [ ]:
img_pos_light = np.array(Image.open('../../../WindingImages/2017-09-10/IMG_2017-09-10_15-55-30.jpg'))
plt.imshow(img_pos_light, cmap='gray')

In [ ]:
img_all = np.array([img_neg_dark, img_neg_light, img_pos_dark, img_pos_light])
img_all_names = np.array(['neg_dark', 'neg_light', 'pos_dark', 'pos_light'])

In [ ]:
means = np.loadtxt('../Means.txt')
stds = np.loadtxt('../StdDev.txt')

In [ ]:
names = ['167_163_200_200', '165_163_200_200', '167_165_163_200_200', '165_165_163_200_200',
        '165_163_163_200_200', '165_163_163_200', '165_165_163_200', '167_165_163_200']

In [ ]:
for img, typ in zip(img_all, img_all_names):

    f = open('../Data/Misc/Timing/2017-09-17/Timing_75_100_{:}.txt'.format(typ), 'a')

    for i, n in enumerate(names):

        model = load_model('../TrainedModels/2017-09-16/{}.h5'.format(n))

        total_res = []
        total_class = []

        # Combine it!

        for i in tqdm(range(10000)):
            # resize and re-shape
            t_start_resize = timer.time()
            img2 = scipy.misc.imresize(img, (h,w))
            # apply normalization
            img2 = (img2-means)/stds
            img2 = np.reshape(img2,[1,h,w,1])
            t_stop_resize = timer.time()

            dt1 = t_stop_resize - t_start_resize
            total_res.append(dt1)

            # classifying
            t_start_class = timer.time()
            Class = model.predict_classes(img2, verbose=0)
            t_stop_class = timer.time()

            dt2 = t_stop_class - t_start_class
            total_class.append(dt2)


        total_res = np.array(total_res)
        total_class = np.array(total_class)

        print('Pic: {}\t Model: {}'.format(typ, n))
        print('Resizing and reshaping {}: {:.8f} +- {:.8f} sec'.format(n, total_res.mean(), total_res.std()))
        print('Classification time {}: {:.8f} +- {:.8f} sec'.format(n, total_class.mean(), total_class.std()))

        f.write('{}\t{:.8f}\t{:.8f}\t{:.8f}\t{:.8f}\n'.format(n, total_res.mean(), total_res.std(), total_class.mean(), total_class.std()))


    f.close()

### dark negative image

In [ ]:
time_path = '../Data/Misc/Timing/2017-09-17/Timing_75_100_neg_dark.txt'

name = np.genfromtxt(time_path, unpack=True, usecols=[0], dtype=str)

res_darkneg, res_std_darkneg, clas_darkneg, clas_std_darkneg = np.genfromtxt(time_path, unpack=True, usecols=[1,2,3,4])

In [ ]:
res_u_darkneg = unumpy.uarray(res_darkneg, res_std_darkneg)
class_u_darkneg = unumpy.uarray(clas_darkneg, res_std_darkneg)
total_u_darkneg = res_u_darkneg+class_u_darkneg

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(x=np.arange(len(clas_darkneg)), y=unumpy.nominal_values(class_u_darkneg)*1000, yerr=unumpy.std_devs(class_u_darkneg)*1000, color='m', marker='.', ls='', label='Classifying')
ax.errorbar(x=np.arange(len(res_darkneg)), y=unumpy.nominal_values(res_u_darkneg)*1000, yerr=unumpy.std_devs(res_u_darkneg)*1000, color='b', marker='.', ls='', label='Resizing')
ax.errorbar(x=np.arange(len(total_u_darkneg)), y=unumpy.nominal_values(total_u_darkneg)*1000, yerr=unumpy.std_devs(total_u_darkneg)*1000, color='c', marker='.', ls='', label='Total')

ax.set_ylabel('Time / ms')
ax.legend(loc='best')
ax.set_title('dark neg img')

_ = plt.xticks(np.arange(len(names)), names, rotation=45, rotation_mode='anchor', ha='right', va='top', fontsize=7)

plt.tight_layout()
plt.savefig('../Data/Misc/Timing/2017-09-17/Times_DarkNegImg.pdf')

### light negative image

In [ ]:
time_path = '../Data/Misc/Timing/2017-09-17/Timing_75_100_neg_light.txt'

name = np.genfromtxt(time_path, unpack=True, usecols=[0], dtype=str)

res_lightneg, res_std_lightneg, clas_lightneg, clas_std_lightneg = np.genfromtxt(time_path, unpack=True, usecols=[1,2,3,4])

In [ ]:
res_u_lightneg = unumpy.uarray(res_lightneg, res_std_lightneg)
class_u_lightneg = unumpy.uarray(clas_lightneg, res_std_lightneg)
total_u_lightneg = res_u_lightneg+class_u_lightneg

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(x=np.arange(len(clas_lightneg)), y=unumpy.nominal_values(class_u_lightneg)*1000, yerr=unumpy.std_devs(class_u_lightneg)*1000, color='m', marker='.', ls='', label='Classifying')
ax.errorbar(x=np.arange(len(res_lightneg)), y=unumpy.nominal_values(res_u_lightneg)*1000, yerr=unumpy.std_devs(res_u_lightneg)*1000, color='b', marker='.', ls='', label='Resizing')
ax.errorbar(x=np.arange(len(total_u_lightneg)), y=unumpy.nominal_values(total_u_lightneg)*1000, yerr=unumpy.std_devs(total_u_lightneg)*1000, color='c', marker='.', ls='', label='Total')

ax.set_ylabel('Time / ms')
ax.legend(loc='best')
ax.set_title('light neg img')

_ = plt.xticks(np.arange(len(names)), names, rotation=45, rotation_mode='anchor', ha='right', va='top', fontsize=7)

plt.tight_layout()
plt.savefig('../Data/Misc/Timing/2017-09-17/Times_LightNegImg.pdf')

### light positive image

In [ ]:
time_path = '../Data/Misc/Timing/2017-09-17/Timing_75_100_pos_light.txt'

name = np.genfromtxt(time_path, unpack=True, usecols=[0], dtype=str)

res_lightpos, res_std_lightpos, clas_lightpos, clas_std_lightpos = np.genfromtxt(time_path, unpack=True, usecols=[1,2,3,4])

In [ ]:
res_u_lightpos = unumpy.uarray(res_lightpos, res_std_lightpos)
class_u_lightpos = unumpy.uarray(clas_lightpos, res_std_lightpos)
total_u_lightpos = res_u_lightpos+class_u_lightpos

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(x=np.arange(len(clas_lightpos)), y=unumpy.nominal_values(class_u_lightpos)*1000, yerr=unumpy.std_devs(class_u_lightpos)*1000, color='m', marker='.', ls='', label='Classifying')
ax.errorbar(x=np.arange(len(res_lightpos)), y=unumpy.nominal_values(res_u_lightpos)*1000, yerr=unumpy.std_devs(res_u_lightpos)*1000, color='b', marker='.', ls='', label='Resizing')
ax.errorbar(x=np.arange(len(total_u_lightpos)), y=unumpy.nominal_values(total_u_lightpos)*1000, yerr=unumpy.std_devs(total_u_lightpos)*1000, color='c', marker='.', ls='', label='Total')

ax.set_ylabel('Time / ms')
ax.legend(loc='best')
ax.set_title('light pos img')

_ = plt.xticks(np.arange(len(names)), names, rotation=45, rotation_mode='anchor', ha='right', va='top', fontsize=7)

plt.tight_layout()
plt.savefig('../Data/Misc/Timing/2017-09-17/Times_LightPosImg.pdf')

### dark positive image

In [ ]:
time_path = '../Data/Misc/Timing/2017-09-17/Timing_75_100_pos_dark.txt'

name = np.genfromtxt(time_path, unpack=True, usecols=[0], dtype=str)

res_darkpos, res_std_darkpos, clas_darkpos, clas_std_darkpos = np.genfromtxt(time_path, unpack=True, usecols=[1,2,3,4])

In [ ]:
res_u_darkpos = unumpy.uarray(res_darkpos, res_std_darkpos)
class_u_darkpos = unumpy.uarray(clas_darkpos, res_std_darkpos)
total_u_darkpos = res_u_darkpos+class_u_darkpos

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(x=np.arange(len(clas_darkpos)), y=unumpy.nominal_values(class_u_darkpos)*1000, yerr=unumpy.std_devs(class_u_darkpos)*1000, color='m', marker='.', ls='', label='Classifying')
ax.errorbar(x=np.arange(len(res_darkpos)), y=unumpy.nominal_values(res_u_darkpos)*1000, yerr=unumpy.std_devs(res_u_darkpos)*1000, color='b', marker='.', ls='', label='Resizing')
ax.errorbar(x=np.arange(len(total_u_darkpos)), y=unumpy.nominal_values(total_u_darkpos)*1000, yerr=unumpy.std_devs(total_u_darkpos)*1000, color='c', marker='.', ls='', label='Total')

ax.set_ylabel('Time / ms')
ax.legend(loc='best')
ax.set_title('dark pos img')

_ = plt.xticks(np.arange(len(names)), names, rotation=45, rotation_mode='anchor', ha='right', va='top', fontsize=7)

plt.tight_layout()
plt.savefig('../Data/Misc/Timing/2017-09-17/Times_DarkPosImg.pdf')

### Total per Net for each img 

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(x=np.arange(len(total_u_darkpos)), y=unumpy.nominal_values(total_u_darkpos)*1000, yerr=unumpy.std_devs(total_u_darkpos)*1000, color='c', marker='.', ls='', label='dark pos')
ax.errorbar(x=np.arange(len(total_u_lightpos)), y=unumpy.nominal_values(total_u_lightpos)*1000, yerr=unumpy.std_devs(total_u_lightpos)*1000, color='m', marker='.', ls='', label='light pos')
ax.errorbar(x=np.arange(len(total_u_darkneg)), y=unumpy.nominal_values(total_u_darkneg)*1000, yerr=unumpy.std_devs(total_u_darkneg)*1000, color='b', marker='.', ls='', label='dark neg')
ax.errorbar(x=np.arange(len(total_u_lightneg)), y=unumpy.nominal_values(total_u_lightneg)*1000, yerr=unumpy.std_devs(total_u_lightneg)*1000, color='k', marker='.', ls='', label='light neg')


ax.set_ylabel('Time / ms')
ax.legend(loc='best')
ax.set_title('total time')

_ = plt.xticks(np.arange(len(names)), names, rotation=45, rotation_mode='anchor', ha='right', va='top', fontsize=7)

plt.tight_layout()
plt.savefig('../Data/Misc/Timing/2017-09-17/Times_Total_allImg.pdf')

# Score distribution for models on all data (normalized)

In [ ]:
# Load test data:

#Creates list of positiv files
positiv_list = glob.glob('/fhgfs/groups/e5/lhcb/detector/scifi/windingcontrol/Images_Sep17/pos/IMG_*.png')
#Finds the number of positiv files
positiv_len = len(positiv_list)
print("Size of positiv Sample: {}".format(positiv_len))

#Same but for negativ files
negativ_list = glob.glob('/fhgfs/groups/e5/lhcb/detector/scifi/windingcontrol/Images_Sep17/neg/IMG_*.png')
negativ_len = len(negativ_list)
print("Size of negativ Sample: {}".format(negativ_len))


X_pos = []
X_neg = []

for fname in tqdm(positiv_list):
    img = scipy.misc.imresize(np.array(Image.open(fname)),(h,w))
    
    X_pos.append(img) 
    
    
for fname in tqdm(negativ_list):
    img = scipy.misc.imresize(np.array(Image.open(fname)),(h,w))
    
    X_neg.append(img)
    
X_pos = np.array(X_pos)
X_neg = np.array(X_neg)

In [ ]:
X_all = np.concatenate((X_pos, X_neg), axis=0)
Y_all = np.concatenate( ( np.ones(len(X_pos)), np.zeros(len(X_neg)) ) , axis=0)

In [ ]:
# Check labeling of Data
if False:
    for i, img in enumerate(X_all[:50]):
        plt.imshow(np.squeeze(img), cmap='gray')

        name = "Good img" if Y_all[i] == 1 else "Bad img"
        plt.title('{}.'.format(name))
        plt.savefig('../Plots/TestImg/{}.png'.format(i))
        if i%10 == 0:
            print("10 written")
        i+=1

In [ ]:
#Shuffling the order of the input - required to avoid batches selecting only positive/negative images
randomize = np.arange(len(X_all))
np.random.shuffle(randomize)
X_all = X_all[randomize]
Y_all = Y_all[randomize]

print(X_all.shape)
print(Y_all.shape)

X_all = X_all.reshape(X_all.shape[0], h, w, 1)

#Shape check (NumberofImages, Height, Width, Depth)
print(X_all.shape)
print(Y_all.shape)

In [ ]:
# Check labeling of Data
if False:
    for i, img in enumerate(X_all[:50]):
        plt.imshow(np.squeeze(img), cmap='gray')

        name = "Good img" if Y_all[i] == 1 else "Bad img"
        plt.title('{}.'.format(name))
        plt.savefig('../Plots/TestImg/{}.png'.format(i))
        if i%10 == 0:
            print("10 written")
        i+=1

### Load Model and corresponding means and stddevs

In [ ]:
# Normalize Imgages

means = np.loadtxt('../TrainedModels/2017-09-15/MeansStdDev/Means_2017-09-15_13-08-59.txt')
stds = np.loadtxt('../TrainedModels/2017-09-15/MeansStdDev/StdDev_2017-09-15_13-08-59.txt')

X_all_norm = np.array([(np.squeeze(img)-means)/stds for img in X_all])

In [ ]:
name = '167_165_163_200'

In [ ]:
# Load Test model:
test_model = load_model('../TrainedModels/2017-09-15/{}.h5'.format(name))

In [ ]:
# Get probs per image
probs = []

for i, img in enumerate(X_all_norm):
    #plt.imshow(np.squeeze(img), cmap='gray')
    img = np.reshape(img, [1,h,w,1])
    p = test_model.predict_proba(img, verbose=0)
    #print('{:}'.format(np.squeeze(p)) )
    probs.append(np.squeeze(p))
    
    #name = "Good img" if Y_test[i, 1] == 1 else "Bad img"
    #plt.title('{}. Score: [{:.4f}, {:.4f}]'.format(name, p[0,0], p[0,1]))
    #plt.savefig('../Plots/TestImg/{}.png'.format(i))
    i+=1

probs = np.array(probs) 

In [ ]:
path_data = '../Data/2017-09-15/{}/'.format(name)

In [ ]:
np.savetxt(path_data + 'ScoresOnAllData.txt', np.transpose([Y_all, probs]))

In [ ]:
# Score
label_pos = Y_all
score_pos = probs[label_pos == 1]
score_neg = probs[label_pos == 0]

In [ ]:
fig, ax = plt.subplots()

binsize = 0.01

bins = np.arange(0, 1 + binsize, binsize)

ax.hist(score_neg, bins=bins, histtype='stepfilled', linewidth=1, edgecolor='m', color='#ff99ff', label='neg img')
ax.hist(score_pos, bins=bins, histtype='stepfilled', linewidth=1, edgecolor='b', color='#6666ff', label='pos img')

ax.hist(score_neg, bins=bins, histtype='step', linewidth=1, linestyle='-', edgecolor='m')
ax.hist(score_pos, bins=bins, histtype='step', linewidth=1, linestyle='-',edgecolor='b')

ax.set_xlabel("Score for pos img")
ax.set_yscale('log')

ax.set_title(name)

ax.legend(loc='best')

plt.tight_layout()
plt.savefig('{}ScoreDistributionOnAllData_{}.png'.format(path_data, binsize))

In [ ]:
# positive images below 0.5
img_pos = X_all_norm[label_pos == 1]
img_pos_false = img_pos[score_pos < 0.5]
score_pos_false = score_pos[score_pos < 0.5]
print(img_pos_false.shape)

# negative images over 0.5
img_neg = X_all_norm[label_pos == 0]
img_neg_false = img_neg[score_neg > 0.5]
score_neg_false = score_neg[score_neg > 0.5]
print(img_neg_false.shape)

In [ ]:
## wrongly classified pics:

for i, (pic,p) in enumerate(zip(img_pos_false[:], score_pos_false[:])):
    plt.imshow(np.squeeze(pic), cmap='gray')
    
    plt.title('Score: {:}'.format(p))
    plt.savefig('{}wrongImg/pos_{}.png'.format(path_data, i))
    #plt.show()

In [ ]:
## wrongly classified pics:

for i, (pic,p) in enumerate(zip(img_neg_false[:], score_neg_false[:])):
    plt.imshow(np.squeeze(pic), cmap='gray')
    
    plt.title('Score: {:}'.format(p))
    plt.savefig('{}/wrongImg/neg_{}.png'.format(path_data, i))
    #plt.show()
    

## ROC Curve

In [ ]:
names

In [ ]:
fig, ax = plt.subplots()

colors = ['springgreen', 'blueviolet', 'firebrick', 'cornflowerblue', 'darkorange', 'aqua', 'deeppink', 'navy']
marker = ['.', 'x', '+', '^', '<', '>', 'd', 'h']

for i, n in enumerate(names):

    Y_all, probs = np.genfromtxt('../Data/2017-09-15/' + n + '/ScoresOnAllData.txt', unpack=True)
    
    auc = roc_auc_score(Y_all, probs)
    #print('AUC: {}'.format(auc))
    
    fpr, tpr, thresholds = roc_curve(Y_all, probs)

    ax.plot(fpr, tpr, color=colors[i], marker=marker[i], markersize=3, ls='--', lw=0.75, label='{} (area={:.4f})'.format(n,auc))


ax.plot([0,1], [0,1], color = 'C7', ls='--',lw=0.75)
    
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

#ax.set_xlim([-0.0001,0.0015])
#ax.set_ylim([0.8, 1.01])

ax.set_title('ROC Curves')

ax.legend(loc='best')

plt.tight_layout()

plt.savefig('../Data/Misc/ROCCurves/2017-09-16/ROC_Curves.pdf')

# Loss and Accuracy during training

In [ ]:
name= '165_163_200_200'

In [ ]:
accuracy, losses = np.genfromtxt('../TrainedModels/2017-09-16/{}_TrainingData.txt'.format(name), unpack=True)

In [ ]:
accuracy = np.array(accuracy)
losses = np.array(losses)

fig, ax = plt.subplots(2, sharex=True)

every_item = 100

ax[0].plot(accuracy[::every_item], color = 'b', marker='', ls='-')
ax[1].plot(losses[::every_item], color = 'm', marker='', ls='-')

ax[1].set_xlabel('batch number')
ax[0].set_ylabel('accuracy')
ax[1].set_ylabel('loss')

ax[1].set_yscale('log')

ax[0].set_title(name)

plt.tight_layout()
plt.savefig('../Data/2017-09-16/{}/LossAccLog::{}.png'.format(name, every_item))

## Score distribution on test data

In [ ]:
Y_test, probs = np.genfromtxt('../Data/2017-09-16/{}/ScoresOnTestData.txt'.format(name), unpack=True)

In [ ]:
label_pos = Y_test
score_pos = probs[label_pos == 1]
score_neg = probs[label_pos == 0]

In [ ]:
fig, ax = plt.subplots()

binsize = 0.05

bins = np.arange(0, 1 + binsize, binsize)

ax.hist(score_neg, bins=bins, histtype='stepfilled', linewidth=1, edgecolor='m', color='#ff99ff', label='neg img')
ax.hist(score_pos, bins=bins, histtype='stepfilled', linewidth=1, edgecolor='b', color='#6666ff', label='pos img')

ax.hist(score_neg, bins=bins, histtype='step', linewidth=1, linestyle='-', edgecolor='m')
ax.hist(score_pos, bins=bins, histtype='step', linewidth=1, linestyle='-',edgecolor='b')

ax.set_xlabel("Score for pos img")
ax.set_yscale('log')

ax.legend(loc='best')

plt.title('Test Data')

plt.tight_layout()
plt.savefig('../Data/2017-09-16/{}/ScoreDistributionOnTestData_{}.png'.format(name, binsize))

# Score distribution for models - live testing

### Static image (no winding)

In [ ]:
score_statimg = np.genfromtxt('../Data/2017-09-10/167_165_163_200_200_selu_100epochs/Probs/WindingProb_2017-09-11_08-08-43_static_image.txt', unpack=True)

In [ ]:
fig, ax = plt.subplots()

binsize = 0.025

bins = np.arange(0, 1 + binsize, binsize)

ax.hist(score_statimg, bins=bins, histtype='stepfilled', linewidth=1, edgecolor='m', color='#ff99ff', label='static postive Image')

ax.set_xlabel("Score")
ax.set_yscale('log')

ax.legend(loc='upper left')

plt.tight_layout()
plt.savefig('../Data/2017-09-10/167_165_163_200_200_selu_100epochs/ScoreDistributionStaticImage_{}.png'.format(binsize))

### winding image (positive)

In [ ]:
score_wind1_pos = np.genfromtxt('../Data/2017-09-10/167_165_163_200_200_selu_100epochs/Probs/WindingProb_2017-09-11_10-03-45.txt')
score_wind2_pos = np.genfromtxt('../Data/2017-09-10/167_165_163_200_200_selu_100epochs/Probs/WindingProb_2017-09-11_10-05-44.txt')
score_wind3_pos = np.genfromtxt('../Data/2017-09-10/167_165_163_200_200_selu_100epochs/Probs/WindingProb_2017-09-11_10-37-26.txt')

In [ ]:
fig, ax = plt.subplots()

binsize = 0.025

bins = np.arange(0, 1 + binsize, binsize)

ax.hist(score_wind1_pos, bins=bins, histtype='stepfilled', linewidth=1, edgecolor='m', color='#ff99ff', label='winding pos Image')
ax.hist(score_wind2_pos, bins=bins, histtype='stepfilled', linewidth=1, edgecolor='b', color='#6666ff', label='pos img')

ax.hist(score_wind1_pos, bins=bins, histtype='step', linewidth=1, linestyle='-', edgecolor='m')
ax.hist(score_wind2_pos, bins=bins, histtype='step', linewidth=1, linestyle='-',edgecolor='b')

ax.set_xlabel("Score")
ax.set_yscale('log')

ax.legend(loc='upper left')

plt.tight_layout()
plt.savefig('../Data/2017-09-10/167_165_163_200_200_selu_100epochs/ScoreDistributionWindingPosImage_{}.png'.format(binsize))